In [25]:
# import torch
# import torch.nn as nn
# from torch.nn import functional as F

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)

# block_size = 8
# batch_size = 4
# max_iters = 10000
# # eval_interval = 2500
# learning_rate = 3e-4

import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250


cuda


In [26]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [27]:
# string_to_int = {ch: i for i, ch in enumerate(chars)}
# int_to_string = {i: ch for i, ch in enumerate(chars)}

# encode = lambda s: [string_to_int[c] for c in s]
# decode = lambda l: ''.join([int_to_string[i] for i in l])

# data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [28]:
# n = int(0.8 * len(data))
# train_data = data[:n]
# val_data = data[n:]

# def get_batch(split):
#     data = train_data if split == 'train' else val_data
#     ix = torch.randint(len(data) - block_size, (batch_size,))
#     x = torch.stack([data[i:i+block_size] for i in ix])
#     y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#     x, y = x.to(device), y.to(device)
#     return x, y

# x, y = get_batch('train')
# print('inputs:')
# print(x)
# print('targets:')
# print(y)
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 0,  3, 36,  1, 76, 64, 65, 70],
        [61, 79,  1, 69, 71, 74, 61,  0],
        [71, 77,  1, 79, 65, 75, 64,  1],
        [71, 81,  1, 68, 71, 71, 67, 61]], device='cuda:0')
targets:
tensor([[ 3, 36,  1, 76, 64, 65, 70, 67],
        [79,  1, 69, 71, 74, 61,  0, 71],
        [77,  1, 79, 65, 75, 64,  1, 62],
        [81,  1, 68, 71, 71, 67, 61, 60]], device='cuda:0')


In [29]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [31]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


!t?*&uT3]xKdxevRkt"Y.OD’sll“j/8)6M bzQwl$%v6Wj9﻿cZ1h’J*zR•omUjt&2P48oBA:3Q”v-T]—cN;c•BWsY0)_&g[2B*HQ5B‘0C”/‘E sD0;RB‘&x3m1Ir'™R8K2q5vD9Veba2i﻿I
Jtx7:o6FvmlekZ")Oz&b4v9
•$A8 ]':,S]q;E_Hn'TefbTM1brwWUcoM﻿tp“sZw““_K‘]qkh-Wi•“4“P)M9tm5j9FJ"YoB]3o]?/f-e bgL([uv;pok'H'tIINS,V&iRt3%DHXw&Yco&JVycR_,4.“?/•4'.B'mhvp—]sDCiO*u‘™)“Y‘"6s8K3Y$X"T()Xh%l?BT]N'5‘fGQ5XhE&2Ier'“*Fs[M[hvwheeP﻿GA 3pIRgowV$cTNQ*]%VgSGT"2I™pba)81‘(Az5Shwf2]“pTvhfx.s﻿1z
uOKospQCu"8?t/
g—q9Dh- O$NjXkvekcr(*:dr'ed™BTk’;RgPMe0X“ViJus]czjU﻿


In [32]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.958, val loss: 4.941
step: 250, train loss: 4.899, val loss: 4.915
step: 500, train loss: 4.816, val loss: 4.854
step: 750, train loss: 4.741, val loss: 4.796
4.53724479675293


In [33]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


pBTCRgS(gFN﻿﻿'BHTA;5IU]•RR;F]gW—
!5uj;jVW&h4cdCRB4•$•xKK‘Jk(Ax3
z.[
L,r'rby)yP6s:].P’f7﻿PVDb™Z*5"j!Jo.UA“u-,wC.o(AllI*G‘/”S1zS9-NMyA•mvnlOXF!d
E•B‘6s!J[X*F’j’K‘Vi7
GI
“?f4FWgF1$BA7/OzxA•coBgPY*”RLWCi“c’llbrw’ah‘3[8$'&0?t)ZifNiWifY_f”;z/OT4'g U?w1‘X&r”6*k“% I™PDY0KEH/Cr'“w'o)o]D2”uvbH;d-gh--.”3w7Hz"M7HSX1‘B‘1,wI$DT4ckp3Mgv-R8)ONc”6pP’?3x3S1pv•CsR1HM8S2”c2l?t[W&bM7; gDH&T4d,—™:C.)OwGFqA9—x﻿L”7c!*2:﻿﻿i’s*Zm4"ps_'﻿3xA W)Iy“Hg™n6CC[Zv/bko“jT4r;*(A")M8&TsD$].]%su,RLOuAlAlqP)h($]3K1tbynk1H”
SXzU7?3xd)B
